In [5]:
import pandas as pd
import numpy as np
import os
import requests

## Sprawdzanie po trans_formed

In [167]:
import os
os.getcwd()

'c:\\Users\\Tomek\\Desktop\\PW MINI IAD\\Semestr 6\\Hurtownie\\ChicagoCrashesETL'

In [202]:
fact_crash = pd.read_pickle('./airflow-postgres-docker/data/tmp/transformed/fact_crash.pkl')
dim_vehicles = pd.read_pickle('./airflow-postgres-docker/data/tmp/transformed/dim_vehicle.pkl')
dim_crash_info = pd.read_pickle('./airflow-postgres-docker/data/tmp/transformed/dim_crash_info.pkl')
fact_weather = pd.read_pickle('./airflow-postgres-docker/data/tmp/transformed/fact_weather.pkl')
dim_people = pd.read_pickle('./airflow-postgres-docker/data/tmp/transformed/dim_people.pkl')
dim_date = pd.read_pickle('./airflow-postgres-docker/data/tmp/transformed/dim_date.pkl')
dim_location = pd.read_pickle('./airflow-postgres-docker/data/tmp/transformed/dim_location.pkl')

In [203]:
# crash_key_map = fact_crash.set_index("CRASH_RECORD_ID")["FACT_CRASH_KEY"]
dim_people_cp = dim_people.copy()

# dim_people_cp.insert(1, "CRASH_KEY", dim_people_cp["CRASH_RECORD_ID"].map(crash_key_map))

fact_crash_expanded = fact_crash.merge(dim_people_cp[["CRASH_RECORD_ID", "PERSON_ID", "VEHICLE_ID"]], on="CRASH_RECORD_ID", how='inner')
fact_crash_expanded

# jeszcze location_id

,FACT_CRASH_KEY,CRASH_RECORD_ID,NUM_UNITS,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,date_id,CRASH_INFO_KEY,LOCATION_KEY,PERSON_ID,VEHICLE_ID
0,1153394593627935104,004cd14d0303a9163aad69a2d7f341b7da2a8572b2ab33...,2,1,1,0,0,0,1,0,2019112609,7017221841570418401,13985346965653409630,O790106,750401
1,1153394593627935104,004cd14d0303a9163aad69a2d7f341b7da2a8572b2ab33...,2,1,1,0,0,0,1,0,2019112609,7017221841570418401,13985346965653409630,O790107,-1
2,913253574156243668,3facd2ed99a27da2f5572605b9434167a6fa7777243e04...,2,1,0,0,1,0,1,0,2019092718,13479415490444100987,6445180670688255903,O747999,710658
3,913253574156243668,3facd2ed99a27da2f5572605b9434167a6fa7777243e04...,2,1,0,0,1,0,1,0,2019092718,13479415490444100987,6445180670688255903,O748000,-1
4,4125836884551549824,4502c89bb17d2a4817c517541350b1470636cc9c29e6fd...,2,0,0,0,0,0,2,0,2019021906,17995927666335292266,9383422351862234978,O587406,560000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263997,1282419967734827564,3a5f25711e9162f0042a53337a25f2c3dc72893c1172d9...,2,0,0,0,0,0,4,0,2019112208,18382781804148259871,6403403058998911966,P180099,747774
263998,1282419967734827564,3a5f25711e9162f0042a53337a25f2c3dc72893c1172d9...,2,0,0,0,0,0,4,0,2019112208,18382781804148259871,6403403058998911966,P180101,747781
263999,1282419967734827564,3a5f25711e9162f0042a53337a25f2c3dc72893c1172d9...,2,0,0,0,0,0,4,0,2019112208,18382781804148259871,6403403058998911966,P182839,758707
264000,1282419967734827564,3a5f25711e9162f0042a53337a25f2c3dc72893c1172d9...,2,0,0,0,0,0,4,0,2019112208,18382781804148259871,6403403058998911966,P182840,758708


In [204]:
dim_location

,LOCATION_KEY,STREET_DIRECTION,STREET_NAME,STREET_NO,LATITUDE,LONGITUDE
0,13985346965653409630,W,TERMINAL ST,5,-999.000000,-999.000000
1,6445180670688255903,S,CALIFORNIA AVE,2517,-999.000000,-999.000000
2,9383422351862234978,W,TERMINAL ST,1,-999.000000,-999.000000
3,380750097830319641,W,TERMINAL ST,1,-999.000000,-999.000000
4,2632866985623213961,W,OHARE ST,1,-999.000000,-999.000000
...,...,...,...,...,...,...
117758,8740512471389969351,N,MILWAUKEE AVE,2357,41.924149,-87.699150
117759,11755592203129826208,W,IRVING PARK RD,3958,41.953758,-87.727478
117760,8682352789340425214,E,LA SALLE DR,150,41.913448,-87.626114
117761,12362181723725837142,W,79TH ST,20,41.751045,-87.625381


In [199]:
dim_date

,date_id,full_date,hour,year,quarter,month,month_name,day_of_month,day_of_week,day_name,is_weekend,week_of_year,is_holiday,holiday_name
0,2016010100,2016-01-01 00:00:00,0,2016,1,1,January,1,5,Friday,False,53,True,New Year's Day
1,2016010101,2016-01-01 01:00:00,1,2016,1,1,January,1,5,Friday,False,53,True,New Year's Day
2,2016010102,2016-01-01 02:00:00,2,2016,1,1,January,1,5,Friday,False,53,True,New Year's Day
3,2016010103,2016-01-01 03:00:00,3,2016,1,1,January,1,5,Friday,False,53,True,New Year's Day
4,2016010104,2016-01-01 04:00:00,4,2016,1,1,January,1,5,Friday,False,53,True,New Year's Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52603,2021123119,2021-12-31 19:00:00,19,2021,4,12,December,31,5,Friday,False,52,True,New Year's Day (observed)
52604,2021123120,2021-12-31 20:00:00,20,2021,4,12,December,31,5,Friday,False,52,True,New Year's Day (observed)
52605,2021123121,2021-12-31 21:00:00,21,2021,4,12,December,31,5,Friday,False,52,True,New Year's Day (observed)
52606,2021123122,2021-12-31 22:00:00,22,2021,4,12,December,31,5,Friday,False,52,True,New Year's Day (observed)


In [200]:
dim_vehicles

,CRASH_UNIT_ID,CRASH_RECORD_ID,UNIT_TYPE,NUM_PASSENGERS,VEHICLE_ID,MAKE,MODEL,VEHICLE_YEAR,VEHICLE_DEFECT,VEHICLE_TYPE,VEHICLE_USE,TRAVEL_DIRECTION,MANEUVER,OCCUPANT_CNT,FIRST_CONTACT_POINT,VEHICLE_KEY
0,1028096,231f7ec818ff8185398af7f7582cc58537d39578c5b279...,DRIVER,-1,974416.0,VOLKSWAGEN,GOLF,-1,NONE,PASSENGER,PERSONAL,SE,STRAIGHT AHEAD,2,OTHER,2477872752993319921
1,1081609,7a51c2f12016df66284d0f660c59c4536b34d63506242b...,DRIVER,-1,1025789.0,JEEP,UKNOWN,2015,UNKNOWN,PASSENGER,PERSONAL,UNKNOWN,TURNING RIGHT,2,OTHER,13315789630448195306
2,1081610,7a51c2f12016df66284d0f660c59c4536b34d63506242b...,DRIVER,-1,1025790.0,HONDA,CIVIC,2017,NONE,PASSENGER,PERSONAL,UNKNOWN,TURNING RIGHT,2,REAR-LEFT,7338186764480988670
3,1094496,5a3aeb04f20bfd350010ca025ad319d7e88eb869875ce9...,DRIVER,-1,1038110.0,HONDA,CONSTRUCTION EQUIPMENT,-1,UNKNOWN,PASSENGER,PERSONAL,N,STRAIGHT AHEAD,2,NONE,2650920976286708066
4,1094497,5a3aeb04f20bfd350010ca025ad319d7e88eb869875ce9...,DRIVER,-1,1038111.0,NISSAN,TRUCK,-1,NONE,PASSENGER,PERSONAL,N,STRAIGHT AHEAD,2,REAR-LEFT,6639772388861697493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240195,1763625,a25e16e62c1e0fd167e184275dfc9a3fe5c02f3d165218...,DRIVER,-1,1679291.0,HONDA,ACCORD,2007,NONE,PASSENGER,PERSONAL,S,BACKING,2,REAR,12526728407230063458
240196,1763626,a25e16e62c1e0fd167e184275dfc9a3fe5c02f3d165218...,DRIVER,1,1679292.0,LEXUS,RX350,2019,NONE,SPORT UTILITY VEHICLE (SUV),PERSONAL,N,BACKING,3,REAR,14760302197810069370
240197,2012713,c1c1c5b94ccf8767e419d2dd6badbc6f0d2ea1ff4224a6...,PARKED,-1,1918147.0,UNKNOWN,OTHER (EXPLAIN IN NARRATIVE),-1,UNKNOWN,UNKNOWN/NA,UNKNOWN/NA,UNKNOWN,PARKED,1,UNKNOWN,13960312683645349882
240198,797065,c1c1c5b94ccf8767e419d2dd6badbc6f0d2ea1ff4224a6...,DRIVER,-1,NaN,UKNOWN,UKNOWN,-1,UKNOWN,UKNOWN,UKNOWN,UKNOWN,UKNOWN,0,UKNOWN,813527030893302580


0

In [9]:
fact_crash = pd.read_pickle("/Users/julia/Desktop/semestr-6/magazyny/ChicagoCrashesETL/airflow-postgres-docker/data/tmp/transformed/fact_crash.pkl")

In [12]:
fact_crash.head(20)

,FACT_CRASH_KEY,CRASH_RECORD_ID,NUM_UNITS,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,date_id
0,102052558975241319,032ca7e09951de4385eec78a50a5f9f33b217838578b29...,3,0,0,0,0,0,1,0,2020010100
1,2989301628946909366,71065ad347ee197080534e112b27c8a5b6fb8d9bbde2c3...,1,2,0,0,2,0,1,0,2020010100
2,15759330137532455643,9c2724305c03bf461d72cbcf2879c96b86ed5e180b62be...,1,-1,-1,-1,-1,-1,-1,-1,2020010100
3,18411092908018610329,3187e5839fab7ff654383ba44a2e5b430c856b3fa10dab...,4,0,0,0,0,0,7,0,2020010100
4,18009739063765005411,449bc7511dba6e6d605411276dc08dbeeccff39b074db0...,2,0,0,0,0,0,1,0,2020010100
5,17528604748918565077,6792e30369a62483f115d4900c9225b17fdca2dc3547f6...,2,0,0,0,0,0,3,0,2020010100
6,4101458230553004886,2cf6f0a893d1cb9f1a0bba9435cd9e1938e2b0a278ffb3...,2,0,0,0,0,0,2,0,2020010100
7,7758129516194055061,9c2ddefdd8c9c409db174e219b0511db3a4c06490e1bcc...,2,0,0,0,0,0,3,0,2020010100
8,14024785583040875057,d100ea227bccd86263d7cbdc1550b24ed1955cffa085ca...,1,0,0,0,0,0,1,0,2020010100
9,16431909946148874362,b73c19ba7e38f0e966410bea19ea5d2d2b75f72b8f2529...,3,0,0,0,0,0,1,0,2020010100
